In [1]:
#%matplotlib inline

In [2]:
#=======================================================================================================================================
# Huggingface load_dataset 으로 MLM 훈련 하기
#
# => load_dataset 으로 wiki 말뭉치를 로딩하고, 이를 토크화 시키고, 
# input_ids 에 대해 15% 확률로 [MASK]를 씌워서, 실제 모델을 훈련시키는 예제 
#
# => MLM 훈련 말뭉치는 bongsoo/moco-corpus-kowiki202206 사용, 평가 말뭉치는 bongsoo/bongevalsmall 사용
#
# 출처 : https://wikidocs.net/166817
#=======================================================================================================================================

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, DistilBertTokenizerFast, BertConfig, DistilBertForMaskedLM

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

#%autosave 900

In [3]:
# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
#input_corpus = "../../data11/my_corpus/my/pre-kowiki-20220620-1줄.txt"
#input_corpus = "bongsoo/moco-corpus"  # huggingface에 등록된 말뭉치 이용
input_corpus = "../../data11/my_corpus/re-kowiki-202206.txt"  #re-kowiki-202206.txt, re-moco-corpus2.txt

# eval 말뭉치 
#eval_corpus = "bongsoo/bongeval"
eval_corpus = "bongsoo/moco_eval"

# 기존 사전훈련된 모델
model_path = "../../data11/model/distilbert/distilbert-base-multilingual-cased"
#model_path = "../../data11/model/distilbert/bert-re-moco-corpus2-mecab"
#model_path = "../../data11/model/distilbert/bert-re-kowiki-moco2-mecab-4"

# 기존 사전 + 추가된 사전 파일
vocab_path = "../../data11/my_corpus/vocab2/bert-re-kowiki202206-mecab-vocab-20000"

# 중간 출력
#OUTPATH = '../../data11/model/distilbert/bert-re-kowiki/'
#OUTPATH = '../../data11/model/distilbert/bert-re-kowiki-nouns/'
OUTPATH = '../../data11/model/distilbert/vocab2/bert-re-kowiki-bert-mecab-check/'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 32       # batch_size (32 이상이면 CUDA MEMORY 부족 함)
token_max_len = 128   # token_seq_len
epoch = 8             # epoch
lr = 5e-5             # learning rate(기본:5e-5)
seed = 111
############################################################################


device = GPU_info()
print(device)

#seed 설정
seed_everything(seed)

#logging 설정
logger =  mlogging(loggername="distilbert-MLM-Trainer", logfilename="../../log/distilbert-MLM-Trainer")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../../log/distilbert-MLM-Trainer_2022-10-29.log


In [4]:
# tokeinzier 생성
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨
tokenizer = DistilBertTokenizerFast.from_pretrained(vocab_path, max_len=token_max_len, strip_accents=False, do_lower_case=False)
#tokenizer = AutoTokenizer.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
# fast 토크너나이즈인지 확인
print(f'{vocab_path} is_fast:{tokenizer.is_fast}')

# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('*special_token_size: {}, *tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('*vocab_size: {}'.format(vocab_size))
print('*tokenizer_len: {}'.format(len(tokenizer)))

# 모델 로딩 further pre-training 
#config = BertConfig.from_pretrained(model_path)
model = DistilBertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
#model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')    

#################################################################################
# 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
# 재설정하지 않으면, 다음과 같은 에러 발생함
# CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
#  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
#
#     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
#     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
#################################################################################
model.resize_token_embeddings(len(tokenizer))

model.to(device)

../../data11/my_corpus/vocab2/bert-re-kowiki202206-mecab-vocab-20000 is_fast:True
*special_token_size: 5, *tokenizer.vocab_size: 139547
*vocab_size: 139548
*tokenizer_len: 139547


DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(139547, 768)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
  

In [5]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 훈련 말뭉치 로딩
#train_dataset = load_dataset(input_corpus)
train_dataset = load_dataset("text", data_files=input_corpus) # text 로컬 파일 로딩

# 평가 말뭉치 로딩
eval_dataset = load_dataset(eval_corpus)

# train_dataset 출력해봄
print(f"train_dataset=======================================")
print(train_dataset)
print(train_dataset['train']['text'][0:3])

print(f'\r\n\r\n')

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset['test']['text'][0:3])

Using custom data configuration default-2734e23cb122ea6a
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/default-2734e23cb122ea6a/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration bongsoo--moco_eval-9741016d12933a55
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--moco_eval-9741016d12933a55/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

train_dataset=======================================
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3714432
    })
})
['스포츠 팀 문화 활동 해외 여행 등의 다양한 과외활동을 제공한다', '남자 자유형 200m 종목의 예선기준 기록은 1분 47초 02이다', '1785년 정조 9 당시 성균관 대사성 민종현 이 왕명에 의해 편찬한 태학지 건치 에 반궁도 가 목판화로 있는데 본 계첩과 30여 년의 시간적 차이가 있어서 그간에 변화된 성균관의 건물을 비교해 볼 수 있다']



eval_dataset========================================
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 3000
    })
})
['필요 시 입력 데이터를 생성하는 외부시스템이 시스템에 접근할 수 있는 FTP 계정과 비밀번호 설정하며 다른 디렉토리에 접근을 막는다.', '외부시스템과 데이터를 주고 받기 위해 사용되는 디렉토리는 다음과 같다.', 'SOA는 비즈니스 프로세스를 기본적인 표준 빌딩 블럭 단위로 분할하여, 이를 IT 프로세스와 유연하게 일치시키는 특징이 있다.']


In [6]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result


# batched=True 하면 빠른 tokenizer 이용(Rust)
%time train_dataset_fast = train_dataset.map(tokenizer_function, batched=True)

%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

'''
%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)
print(tokenized_dataset_fast['train']['text'][0:2])
'''

  0%|          | 0/3715 [00:00<?, ?ba/s]

Loading cached processed dataset at /MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--moco_eval-9741016d12933a55/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4/cache-87caaf4e765ab88a.arrow


CPU times: user 11min 18s, sys: 21min 59s, total: 33min 17s
Wall time: 2min 21s
CPU times: user 33.8 ms, sys: 5.22 ms, total: 39 ms
Wall time: 38.9 ms


"\n%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)\nprint(tokenized_dataset_fast['train']['text'][0:2])\n"

In [7]:
print(f"train_dataset_fast=======================================")
print(train_dataset_fast)
print(f'*fast_len:{len(train_dataset_fast["train"])}, len:{len(train_dataset["train"])}')  # fast_dataset과 dataset 길이를 비교함
print(train_dataset_fast['train'][0:2])

print(f'\r\n\r\n')

print(f"eval_dataset_fast=======================================")
print(eval_dataset_fast)
print(f'*fast_len:{len(eval_dataset_fast["test"])}, len:{len(eval_dataset["test"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['test'][0:2])

train_dataset_fast=======================================
DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 3717139
    })
})
*fast_len:3717139, len:3714432
{'text': ['스포츠 팀 문화 활동 해외 여행 등의 다양한 과외활동을 제공한다', '남자 자유형 200m 종목의 예선기준 기록은 1분 47초 02이다'], 'input_ids': [[101, 120257, 9899, 119594, 119570, 120648, 120412, 28697, 53645, 8898, 78705, 119446, 69448, 119692, 14102, 102], [101, 76854, 132874, 10777, 10147, 120740, 10459, 120472, 12310, 54867, 119566, 10892, 122, 37712, 11413, 57030, 10983, 11925, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}



eval_dataset_fast=======================================
DatasetDict({
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 3002
    })
})
*fast_len:3002, len:3000
{'text': ['필요 시 입력 데이터를 생성하는 외부시스템이 시스템에 접근할 수 있는 FTP 계정과 비밀번호 설정하며 다른 디렉토리에 접근을 막는다.', 

In [8]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_train_sample = data_collator(train_dataset_fast['train']['input_ids'][0:2])
mlm_eval_sample = data_collator(eval_dataset_fast['test']['input_ids'][0:2])

print(f"train_dataset_fast(MLM)=======================================")
print(mlm_train_sample['input_ids'][0])
print(train_dataset_fast['train'][0])

print(f'\r\n\r\n')

print(f"eval_dataset_fast(MLM)=======================================")
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast['test'][0])

train_dataset_fast(MLM)=======================================
tensor([   101,    103,   9899,    103, 119570, 120648, 120412,  28697,    103,
          8898,  78705, 119446,  69448, 119692,  14102,    102,      0,      0,
             0])
{'text': '스포츠 팀 문화 활동 해외 여행 등의 다양한 과외활동을 제공한다', 'input_ids': [101, 120257, 9899, 119594, 119570, 120648, 120412, 28697, 53645, 8898, 78705, 119446, 69448, 119692, 14102, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}



eval_dataset_fast(MLM)=======================================
tensor([   101, 119649,   9485, 122248, 119997,  11513, 120350,    103, 120597,
        121779, 108366,  10739, 119725,  10530, 120196,  14843,   9460,  13767,
           143,    103, 125736,  11882, 120733,  35465,  20309, 120551,  22766,
         19709, 126329, 120565,  10530, 120196,  10622,   9247,  40410,    119,
           102])
{'text': '필요 시 입력 데이터를 생성하는 외부시스템이 시스템에 접근할 수 있는 FTP 계정과 비밀번호 설정하며 다른 디렉토리에 접근을 막는다.', 'input_ids': [101, 119649, 

In [9]:
# 훈련 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#########################################################################################
# hyper parameter 설정
#########################################################################################

epochs = epoch          # epochs

total_optim_steps = len(train_dataset_fast["train"]) * epochs // batch_size   # 총 optimize(역전파) 스탭수 = 훈련dataset 계수 * epochs // 배치 크기
eval_steps=int(total_optim_steps * 0.02)           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)
save_steps=int(total_optim_steps * 0.1)            # 저장 스탭수 
#save_total_limit=5                                # 마지막 5개 남기고 삭제 

print(f'*total_optim_steps: {total_optim_steps}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}, *save_steps:{save_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                      # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    overwrite_output_dir=True,         
    num_train_epochs=epochs,                # 에폭
    learning_rate=lr,                       # lr: 기본 5e-5
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    save_strategy="steps",                  # 저장 전략 (no, epoch, steps 기본=steps) 
    save_steps=save_steps,                  # step 수마다 모델을 저장
    #save_total_limit=save_total_limit,     # 마지막 x개 모델 빼고 과거 모델은 삭제
    evaluation_strategy="steps",            # 평가 전략 (no, epoch, steps 기본=no)  
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
train_dataset_fast_input_ids = train_dataset_fast['train']['input_ids']
eval_dataset_fast_input_ids = eval_dataset_fast['test']['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    train_dataset=train_dataset_fast_input_ids,   # 훈련 데이터셋
    eval_dataset=eval_dataset_fast_input_ids      # 평가 데이터셋
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


*total_optim_steps: 929284, *eval_steps:18585, *logging_steps:18585, *save_steps:92928
*no_cuda: False


Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
# 훈련 시작
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3717139
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 929288
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss,Validation Loss
18585,4.354000,4.799882
37170,3.619100,4.605822
55755,3.392800,4.649975
74340,3.264800,4.578352
92925,3.178100,4.514098
111510,3.112500,4.460934
130095,3.044800,4.420611
148680,3.004200,4.456695
167265,2.965700,4.421189
185850,2.931900,4.426220


***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
Saving model checkpoint to ../../data11/model/distilbert/vocab2/bert-re-kowiki-bert-mecab-check/checkpoint-92928
Configuration saved in ../../data11/model/distilbert/vocab2/bert-re-kowiki-bert-mecab-check/checkpoint-92928/config.json
Model weights saved in ../../data11/model/distilbert/vocab2/bert-re-kowiki-bert-mecab-check/checkpoint-92928/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Running Evaluation *****
  Num examples = 3002
  Batch size = 8
***** Run

In [ ]:
# 모델 저장
### 전체모델 저장
TMP_OUT_PATH = '../../data11/model/distilbert/vocab2/bert-re-kowiki-bert-mecab/'

os.makedirs(TMP_OUT_PATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(TMP_OUT_PATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = TMP_OUT_PATH
tokenizer.save_pretrained(VOCAB_PATH)
print(f'==> save_model : {TMP_OUT_PATH}')